<a href="https://colab.research.google.com/github/vilsonrodrigues/WebScraping/blob/master/Extracao_Links_Boletins_Brasil_IO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Contexto: 

###O site Brasil IO disponibiliza uma série de links dos boletins das secretarías de saúde por estado.

#Problema:

###Esses links estão como hiperlinks, não estão direto dentro da tabela, o desafio é retirar esses links específicos de dentro da Tag TD.

#1. Bibliotecas

###1.1 Instalação

In [0]:
!pip install numpy
!pip install pandas
!pip install beautifulsoup4

###1.2 Importação

In [0]:
import pandas as pd
import urllib3
import datetime
from datetime import timedelta
from bs4 import BeautifulSoup

#2. Formação da Base de Dados dos Links


Objeto da urllib3 para extração já com o Proxy configurado

In [0]:
http = urllib3.PoolManager()

Obtendo a data anterior para realizar a consulta no site do Brasil IO

In [3]:
dia_anterior = (datetime.datetime.now() - timedelta(days=1))
dia_anterior = dia_anterior.strftime('%Y-%m-%d')
dia_anterior

'2020-04-20'

Criação do link adaptável conforme o dia

In [4]:
url = 'https://brasil.io/dataset/covid19/boletim?search=&date=' + dia_anterior + '&state='
url

'https://brasil.io/dataset/covid19/boletim?search=&date=2020-04-20&state='

Realizando Requisão GET e verificando se deu certo

In [5]:
req = http.request('GET',url)
print(req.status)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200


Usando soup para ler o objeto no formato HTML

In [0]:
soup = BeautifulSoup(req.data, 'html.parser')

Lendo com o Pandas a tabela explícita

In [7]:
table_str = str(soup)
df = pd.read_html(table_str)[0]
df = df.iloc[:,:2]
df.columns = ['date','uf']
df

,date,uf
0,2020-04-20,AC
1,2020-04-20,AC
2,2020-04-20,AL
3,2020-04-20,AM
4,2020-04-20,AP
5,2020-04-20,BA
6,2020-04-20,CE
7,2020-04-20,DF
8,2020-04-20,ES
9,2020-04-20,MA


Retirando os links de dentro da tag TD

In [8]:
links = []
tabela_td = soup.select('td a')
for a in tabela_td:
  print(a['href'])
  links.append(a['href'])

print('Total de links', len(links))

https://agencia.ac.gov.br/boletim-sesacre-desta-segunda-feira-20-sobre-o-coronavirus/
https://agencia.ac.gov.br/wp-content/uploads/2020/04/BOLETIM_COVID-19_ACRE_31_20.04.2020-1.pdf
http://www.saude.al.gov.br/2020/04/20/boletim-20-04-al-tem-171-casos-da-covid-19-e-18-obitos/
http://www.saude.am.gov.br/visualizar-noticia.php?id=4473
https://www.portal.ap.gov.br/noticia/2004/atualizacao-ndash-boletim-informativo-covid-19-amapa-20-de-abril-de-2020-ndash-18h30
http://www.saude.ba.gov.br/2020/04/20/bahia-tem-1-377-confirmacoes-de-covid-19/
https://indicadores.integrasus.saude.ce.gov.br/indicadores/indicadores-coronavirus/coronavirus-ceara
http://www.saude.df.gov.br/wp-conteudo/uploads/2020/03/Boletim-COVID_DF-20.04.2020_prisional-separado.pdf
https://coronavirus.es.gov.br/painel-covid-19-es
http://www.saude.ma.gov.br/wp-content/uploads/2020/04/nota41.pdf
https://www.saude.mg.gov.br/images/noticias_e_eventos/000_2020/Boletins_Corona/20.04.2020_Boletim_epidemiologico_COVID-19_MG.pdf
https://ww

Inserindo a lista de links no DF

In [9]:
df['link'] = links
df.head() 

,date,uf,link
0,2020-04-20,AC,https://agencia.ac.gov.br/boletim-sesacre-dest...
1,2020-04-20,AC,https://agencia.ac.gov.br/wp-content/uploads/2...
2,2020-04-20,AL,http://www.saude.al.gov.br/2020/04/20/boletim-...
3,2020-04-20,AM,http://www.saude.am.gov.br/visualizar-noticia....
4,2020-04-20,AP,https://www.portal.ap.gov.br/noticia/2004/atua...


Removendo campos onde seja uma duplicata, ou seja, no dia saiu mais uma atualização, mantenho ela

In [0]:
df = df.drop_duplicates(subset = ['uf'], keep = 'last')

In [11]:
df

,date,uf,link
1,2020-04-20,AC,https://agencia.ac.gov.br/wp-content/uploads/2...
2,2020-04-20,AL,http://www.saude.al.gov.br/2020/04/20/boletim-...
3,2020-04-20,AM,http://www.saude.am.gov.br/visualizar-noticia....
4,2020-04-20,AP,https://www.portal.ap.gov.br/noticia/2004/atua...
5,2020-04-20,BA,http://www.saude.ba.gov.br/2020/04/20/bahia-te...
6,2020-04-20,CE,https://indicadores.integrasus.saude.ce.gov.br...
7,2020-04-20,DF,http://www.saude.df.gov.br/wp-conteudo/uploads...
8,2020-04-20,ES,https://coronavirus.es.gov.br/painel-covid-19-es
9,2020-04-20,MA,http://www.saude.ma.gov.br/wp-content/uploads/...
10,2020-04-20,MG,https://www.saude.mg.gov.br/images/noticias_e_...


Persistindo o DataFrame em disco

In [0]:
dt.to_csv('links_boletins.csv')